In [32]:
# run stopwatch
from tools import Stopwatch
stopwatch = Stopwatch()
start = stopwatch.start()

### Load Saved Section if exists

### Load libraries, functions, palette, theme

In [33]:
%run load.ipynb

In [34]:
# read file from another directory
path = \
    ('/Users/achasovsky/Library/CloudStorage/GoogleDrive-alexanderchasovsky@gmail.com/'
     'Мой диск/Data Science Projects/# economy-rus/data/')

economics_data = loadit(
    'economics_data', path + '00-Datasets/'
)

In [35]:
dir = '009.4-009.8_PI-Consumers-Indexes'

In [112]:
save_df = path + '/00-Datasets/'

In [103]:
save_excel = path + '/00-Excel-data/'

# <font color='#2C8B6D'>II. Цены, инфляция</font>

# Section I. Потребительские цены

## 9. Индексы потребительских цен

### 9.4. Индексы потребительских цен на товары и услуги по Российской Федерации, федеральным округам и субъектам Российской Федерации (с 1992 г.)

*Источник: https://rosstat.gov.ru/statistics/price*  
*Единицы измерения: *

*Примечания:*  

In [38]:
pci_link = \
    'https://rosstat.gov.ru/storage/mediabank/ipc_s_1992.xlsx'

In [39]:
pci_raw = pd.read_excel(pci_link)

In [40]:
pci_raw

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,NaN,ИНДЕКСЫ ПОТРЕБИТЕЛЬСКИХ ЦЕН\nна все товары и у...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"на конец периода, в % к декабрю предыдущего года",NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1992,1993.0,1994.00,1995.0,1996.00,1997.00,1998.00,1999.00,2000.00,...,2013.00,2014.00,2015.00,2016,2017.00,2018.00,2019.00,2020.00,2021.00,2022.00
3,Российская Федерация,2608.84,939.9,315.14,231.3,121.81,111.03,184.43,136.53,120.18,...,106.47,111.35,112.91,105.39,102.51,104.26,103.04,104.91,108.39,111.94
4,Центральный федеральный округ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.71,...,106.68,111.99,113.68,105.85,103.18,104.65,103.01,104.70,108.32,112.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,2) до 2005 г. Пермская область и Коми-Пермяцки...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,"3) до 2007 г. Красноярский край, Таймырский (...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,4) до 2008 г. Иркутская область и Усть-Ордынск...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,5) до 2008 г. Читинская область и Агинский Бу...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
pci = pci_raw.iloc[2:-7, :].copy()

In [42]:
pci.iloc[0, 0] = 'Регион'

In [43]:
pci.columns = pci.iloc[0, :]
pci = pci.drop(2, axis=0)

In [44]:
pci.columns = [to_int(i) for i in pci.columns]
pci.columns.name = None

In [45]:
pci['Регион'] = [i.strip() for i in pci['Регион']]

In [46]:
replace_list = ['1)', '2)', '3)', '4)', '5)', '6)']

In [47]:
pci['Регион'] = [replace_elements(i, '', replace_list) for i in pci['Регион']]

In [48]:
astrakhan_names = [
    'Архангельская область (кроме Ненецкого автономного округа)', 'Архангельская область']
pci = sum_rows( pci, 'Регион', astrakhan_names, 'Архангельская область без автономных округов')

In [49]:
tyumen_names = [
    'Тюменская область (кроме Ханты-Мансийского автономного округа - Югры и Ямало-Ненецкого автономного округа)',
    'Тюменская область']
pci = sum_rows(pci, 'Регион', tyumen_names, 'Тюменская область без автономных округов')

In [50]:
pci = pci[~pci['Регион'].isin(federal_districts_names_list)].copy()

In [51]:
pci = pci[~pci['Регион'].isin(regions_drop_list)].copy()

In [52]:
pci['Регион'] = pci['Регион'].replace(regions_replace_dict, regex=True)
pci['Регион'] = pci['Регион'].replace(regions_replace_dict)

In [53]:
pci = pci.T.copy()

In [54]:
pci.columns = pci.loc['Регион']
pci = pci.drop('Регион', axis=0)
pci.columns.name = None

In [55]:
pci = pci.astype(float)

In [56]:
pci = transform_round(pci, 1)

In [57]:
isna(pci.T)

,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Республика Крым,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,142.5,127.6,107.2,101.4,105.5,102.6,105.2,108.7,113.2
Севастополь,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,136.1,121.0,106.5,103.8,105.1,102.5,105.6,110.1,112.1
Республика Ингушетия,NaN,NaN,337.2,196.0,117.4,106.9,172.8,152.3,122.7,138.7,...,104.5,108.1,117.5,102.6,104.0,101.9,103.4,105.8,108.9,118.6
Чеченская Республика,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,106.2,111.1,114.8,105.2,102.0,103.5,103.0,105.0,107.6,112.9


In [58]:
pci.head()

,Российская Федерация,Белгородская область,Брянская область,Владимирская область,Воронежская область,Ивановская область,Калужская область,Костромская область,Курская область,Липецкая область,...,Республика Саха,Забайкальский край,Камчатский край,Приморский край,Хабаровский край,Амурская область,Магаданская область,Сахалинская область,Еврейская авт.область,Чукотский авт.округ
1992,2608.8,2601.9,1758.8,2349.4,2584.7,2780.9,2258.7,2866.8,2425.8,1998.0,...,2627.6,3115.4,4896.5,2883.9,3063.1,2654.0,5265.0,3185.8,2138.6,2551.0
1993,939.9,791.9,810.7,912.4,824.3,829.3,874.9,828.2,960.3,870.6,...,1008.8,1359.3,1643.0,1130.0,1123.3,1203.5,1163.0,1718.4,1520.0,2058.0
1994,315.1,316.4,325.6,313.7,300.6,331.5,336.2,323.1,344.1,319.5,...,257.7,305.2,305.5,309.9,306.4,291.9,249.0,365.2,318.6,343.0
1995,231.3,228.4,261.4,240.2,240.4,238.3,256.9,245.1,251.1,256.3,...,232.5,236.8,239.2,222.3,218.2,218.4,200.4,239.0,244.6,247.0
1996,121.8,129.2,119.2,117.5,129.8,124.4,120.5,121.0,120.0,122.6,...,121.8,121.1,127.3,122.2,120.6,122.1,121.3,131.8,125.5,156.0


### Индексы потребительских цен на товары и услуги по Российской Федерации, месяцы (с 1991 г.)

*Источник: https://rosstat.gov.ru/statistics/price*  
*Единицы измерения: *

*Примечания:*  

In [59]:
pci_month_link = \
    'https://rosstat.gov.ru/storage/mediabank/ipc_mes_03-2024.xlsx'

### 9.5. Индексы потребительских цен на <font color='#2C8B6D'> товары и услуги </font> по Российской Федерации в 1991-2023 гг.

In [60]:
pci_month_raw = pd.read_excel(pci_month_link, sheet_name='01')

In [61]:
pci_month_raw

,Индексы потребительских цен на товары и услуги1) по Российской Федерации в 1991-2024*)гг.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
0,К содержанию,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"на конец периода, в %",NaN,NaN
2,NaN,1991.0,1992.00,1993.00,1994.00,1995.00,1996.00,1997.00,1998.00,1999.00,...,2015.00,2016.00,2017.00,2018.00,2019.00,2020.00,2021.00,2022,2023.00,2024
3,к концу предыдущего месяца,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,январь,106.2,345.30,125.80,117.90,117.77,104.11,102.34,101.51,108.38,...,103.85,100.96,100.62,100.31,101.01,100.40,100.67,100.99,100.84,100.86
5,февраль,104.8,138.00,124.70,110.82,111.02,102.79,101.54,100.89,104.13,...,102.22,100.63,100.22,100.21,100.44,100.33,100.78,101.17,100.46,100.68
6,март,106.3,129.90,120.10,107.41,108.94,102.80,101.43,100.64,102.79,...,101.21,100.46,100.13,100.29,100.32,100.55,100.66,107.61,100.37,100.39
7,апрель,163.5,121.70,118.70,108.49,108.47,102.16,100.96,100.38,103.03,...,100.46,100.44,100.33,100.38,100.29,100.83,100.58,101.56,100.38,NaN
8,май,103.0,111.90,118.10,106.91,107.93,101.60,100.94,100.50,102.22,...,100.35,100.41,100.37,100.38,100.34,100.27,100.74,100.12,100.31,NaN
9,июнь,101.2,119.10,119.90,106.00,106.66,101.17,101.10,100.08,101.91,...,100.19,100.36,100.61,100.49,100.04,100.22,100.69,99.65,100.37,NaN


In [62]:
pci_month = pci_month_raw[2:-3].copy()

In [63]:
pci_month.iloc[0,0] = '-'

In [64]:
pci_month.columns = pci_month.iloc[0, :]

In [65]:
pci_month = pci_month[2:].copy()

In [66]:
pci_month.columns = [to_int(i) for i in pci_month.columns]

In [67]:
pci_month = pci_month.drop(16, axis=0)

In [68]:
pci_month.loc[17, '-'] = 'Год'

In [69]:
pci_month = pci_month.set_index('-', drop=True)

In [70]:
pci_month.index.name = None

In [71]:
replace_dict = {'1)': '', '2)': '', ',': '.'}

In [72]:
pci_month = pci_month.astype(str)

In [73]:
for column in pci_month.columns:
    for key, value in replace_dict.items():
        pci_month[column] = pci_month[column].str.replace(key, value)

In [74]:
pci_month = pci_month.astype(float)

In [75]:
pci_month = transform_round(pci_month, 1)

In [76]:
isna(pci_month)

,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
апрель,163.5,121.7,118.7,108.5,108.5,102.2,101.0,100.4,103.0,100.9,...,100.5,100.4,100.3,100.4,100.3,100.8,100.6,101.6,100.4,NaN
май,103.0,111.9,118.1,106.9,107.9,101.6,100.9,100.5,102.2,101.8,...,100.4,100.4,100.4,100.4,100.3,100.3,100.7,100.1,100.3,NaN
июнь,101.2,119.1,119.9,106.0,106.7,101.2,101.1,100.1,101.9,102.6,...,100.2,100.4,100.6,100.5,100.0,100.2,100.7,99.6,100.4,NaN
июль,100.6,110.6,122.4,105.3,105.4,100.7,100.9,100.2,102.8,101.8,...,100.8,100.5,100.1,100.3,100.2,100.4,100.3,99.6,100.6,NaN
август,100.5,108.6,126.0,104.6,104.6,99.8,99.9,103.7,101.2,101.0,...,100.4,100.0,99.5,100.0,99.8,100.0,100.2,99.5,100.3,NaN
сентябрь,101.1,111.5,123.0,108.0,104.5,100.3,99.7,138.4,101.5,101.3,...,100.6,100.2,99.8,100.2,99.8,99.9,100.6,100.0,100.9,NaN
октябрь,103.5,122.9,119.5,115.0,104.7,101.2,100.2,104.5,101.4,102.1,...,100.7,100.4,100.2,100.4,100.1,100.4,101.1,100.2,100.8,NaN
ноябрь,108.9,126.1,116.4,114.6,104.6,101.9,100.6,105.7,101.2,101.5,...,100.8,100.4,100.2,100.5,100.3,100.7,101.0,100.4,101.1,NaN
декабрь,112.1,125.2,112.5,116.4,103.2,101.4,101.0,111.6,101.3,101.6,...,100.8,100.4,100.4,100.8,100.4,100.8,100.8,100.8,100.7,NaN


In [77]:
# extract last row with 'Год' from 'pci_month'
first_row = pci_month.iloc[-1].to_frame().T

In [78]:
# add it as first row to 'pci_month'
pci_month = pd.concat([first_row, pci_month], axis=0)

In [79]:
# remove last row from 'pci_month'
pci_month = pci_month.iloc[:-1]

In [80]:
pci_month_rus = pci_month.copy()

In [119]:
pci_month_rus.index = [i.capitalize() for i in pci_month_rus.index]

In [120]:
pci_month_rus

,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Год,260.4,2608.8,939.9,315.1,231.3,121.8,111.0,184.4,136.5,120.2,...,112.9,105.4,102.5,104.3,103.0,104.9,108.4,111.9,107.4,102.0
Январь,106.2,345.3,125.8,117.9,117.8,104.1,102.3,101.5,108.4,102.3,...,103.8,101.0,100.6,100.3,101.0,100.4,100.7,101.0,100.8,100.9
Февраль,104.8,138.0,124.7,110.8,111.0,102.8,101.5,100.9,104.1,101.0,...,102.2,100.6,100.2,100.2,100.4,100.3,100.8,101.2,100.5,100.7
Март,106.3,129.9,120.1,107.4,108.9,102.8,101.4,100.6,102.8,100.6,...,101.2,100.5,100.1,100.3,100.3,100.6,100.7,107.6,100.4,100.4
Апрель,163.5,121.7,118.7,108.5,108.5,102.2,101.0,100.4,103.0,100.9,...,100.5,100.4,100.3,100.4,100.3,100.8,100.6,101.6,100.4,NaN
Май,103.0,111.9,118.1,106.9,107.9,101.6,100.9,100.5,102.2,101.8,...,100.4,100.4,100.4,100.4,100.3,100.3,100.7,100.1,100.3,NaN
Июнь,101.2,119.1,119.9,106.0,106.7,101.2,101.1,100.1,101.9,102.6,...,100.2,100.4,100.6,100.5,100.0,100.2,100.7,99.6,100.4,NaN
Июль,100.6,110.6,122.4,105.3,105.4,100.7,100.9,100.2,102.8,101.8,...,100.8,100.5,100.1,100.3,100.2,100.4,100.3,99.6,100.6,NaN
Август,100.5,108.6,126.0,104.6,104.6,99.8,99.9,103.7,101.2,101.0,...,100.4,100.0,99.5,100.0,99.8,100.0,100.2,99.5,100.3,NaN
Сентябрь,101.1,111.5,123.0,108.0,104.5,100.3,99.7,138.4,101.5,101.3,...,100.6,100.2,99.8,100.2,99.8,99.9,100.6,100.0,100.9,NaN


In [82]:
with pd.option_context('display.max_rows', 15):
    display(pci_month_rus)

,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Год,260.4,2608.8,939.9,315.1,231.3,121.8,111.0,184.4,136.5,120.2,...,112.9,105.4,102.5,104.3,103.0,104.9,108.4,111.9,107.4,102.0
январь,106.2,345.3,125.8,117.9,117.8,104.1,102.3,101.5,108.4,102.3,...,103.8,101.0,100.6,100.3,101.0,100.4,100.7,101.0,100.8,100.9
февраль,104.8,138.0,124.7,110.8,111.0,102.8,101.5,100.9,104.1,101.0,...,102.2,100.6,100.2,100.2,100.4,100.3,100.8,101.2,100.5,100.7
март,106.3,129.9,120.1,107.4,108.9,102.8,101.4,100.6,102.8,100.6,...,101.2,100.5,100.1,100.3,100.3,100.6,100.7,107.6,100.4,100.4
апрель,163.5,121.7,118.7,108.5,108.5,102.2,101.0,100.4,103.0,100.9,...,100.5,100.4,100.3,100.4,100.3,100.8,100.6,101.6,100.4,NaN
май,103.0,111.9,118.1,106.9,107.9,101.6,100.9,100.5,102.2,101.8,...,100.4,100.4,100.4,100.4,100.3,100.3,100.7,100.1,100.3,NaN
июнь,101.2,119.1,119.9,106.0,106.7,101.2,101.1,100.1,101.9,102.6,...,100.2,100.4,100.6,100.5,100.0,100.2,100.7,99.6,100.4,NaN
июль,100.6,110.6,122.4,105.3,105.4,100.7,100.9,100.2,102.8,101.8,...,100.8,100.5,100.1,100.3,100.2,100.4,100.3,99.6,100.6,NaN
август,100.5,108.6,126.0,104.6,104.6,99.8,99.9,103.7,101.2,101.0,...,100.4,100.0,99.5,100.0,99.8,100.0,100.2,99.5,100.3,NaN
сентябрь,101.1,111.5,123.0,108.0,104.5,100.3,99.7,138.4,101.5,101.3,...,100.6,100.2,99.8,100.2,99.8,99.9,100.6,100.0,100.9,NaN


In [83]:
pci_month.index = ['Year'] + [months_translate(x) for x in pci_month.index[1:]]

In [84]:
pci_month

,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Year,260.4,2608.8,939.9,315.1,231.3,121.8,111.0,184.4,136.5,120.2,...,112.9,105.4,102.5,104.3,103.0,104.9,108.4,111.9,107.4,102.0
January,106.2,345.3,125.8,117.9,117.8,104.1,102.3,101.5,108.4,102.3,...,103.8,101.0,100.6,100.3,101.0,100.4,100.7,101.0,100.8,100.9
February,104.8,138.0,124.7,110.8,111.0,102.8,101.5,100.9,104.1,101.0,...,102.2,100.6,100.2,100.2,100.4,100.3,100.8,101.2,100.5,100.7
March,106.3,129.9,120.1,107.4,108.9,102.8,101.4,100.6,102.8,100.6,...,101.2,100.5,100.1,100.3,100.3,100.6,100.7,107.6,100.4,100.4
April,163.5,121.7,118.7,108.5,108.5,102.2,101.0,100.4,103.0,100.9,...,100.5,100.4,100.3,100.4,100.3,100.8,100.6,101.6,100.4,NaN
May,103.0,111.9,118.1,106.9,107.9,101.6,100.9,100.5,102.2,101.8,...,100.4,100.4,100.4,100.4,100.3,100.3,100.7,100.1,100.3,NaN
June,101.2,119.1,119.9,106.0,106.7,101.2,101.1,100.1,101.9,102.6,...,100.2,100.4,100.6,100.5,100.0,100.2,100.7,99.6,100.4,NaN
July,100.6,110.6,122.4,105.3,105.4,100.7,100.9,100.2,102.8,101.8,...,100.8,100.5,100.1,100.3,100.2,100.4,100.3,99.6,100.6,NaN
August,100.5,108.6,126.0,104.6,104.6,99.8,99.9,103.7,101.2,101.0,...,100.4,100.0,99.5,100.0,99.8,100.0,100.2,99.5,100.3,NaN
September,101.1,111.5,123.0,108.0,104.5,100.3,99.7,138.4,101.5,101.3,...,100.6,100.2,99.8,100.2,99.8,99.9,100.6,100.0,100.9,NaN


### 9.6. Индексы потребительских цен на <font color='#2C8B6D'> продовольственные товары </font> по Российской Федерации в 1991-2023 гг.

In [85]:
pci_food_raw = pd.read_excel(pci_month_link, sheet_name='02')

In [86]:
pci_food_raw.tail(20)

,Индексы потребительских цен на продовольственные товары по Российской Федерации в 1991-2024*)гг.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"на конец периода, в %",NaN,NaN
2,NaN,1991.0,1992.00,1993.00,1994.00,1995.00,1996.00,1997.00,1998.00,1999.00,...,2015.00,2016.00,2017.00,2018.00,2019.00,2020.00,2021.00,2022,2023.00,2024
3,к концу предыдущего месяца,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,январь,104.6,405.10,129.00,121.00,121.05,103.95,103.05,102.07,110.30,...,105.66,101.19,100.85,100.50,101.27,100.69,101.01,101.44,101.32,101.26
5,февраль,103.1,129.80,124.78,110.60,110.27,102.40,101.42,101.22,104.37,...,103.29,100.68,100.20,100.35,100.79,100.57,101.22,101.56,100.79,100.77
6,март,105.2,123.00,118.30,105.80,108.46,102.92,101.42,100.72,102.79,...,101.57,100.40,100.14,100.53,100.53,100.96,100.82,106.73,100.13,100.17
7,апрель,171.8,116.11,116.10,105.60,107.52,101.84,100.98,100.32,102.57,...,100.33,100.42,100.60,100.43,100.43,101.72,100.75,102.87,100.29,NaN
8,май,100.5,110.70,116.20,106.80,108.83,101.66,100.81,100.58,102.01,...,100.07,100.37,100.59,99.94,100.41,100.16,100.96,100.6,99.69,NaN
9,июнь,99.7,120.00,122.70,105.60,107.56,101.20,101.47,99.98,101.72,...,99.55,100.12,101.02,100.38,99.52,100.17,100.64,98.9,99.99,NaN
10,июль,97.2,108.90,122.51,103.80,104.47,100.21,100.79,99.90,103.18,...,99.73,99.96,99.03,99.67,99.70,99.93,99.50,98.47,100.49,NaN


In [87]:
pci_food = transform_pci(pci_food_raw)

In [88]:
pci_food.index = ['Year'] + [months_translate(x) for x in pci_food.index[1:]]

In [89]:
pci_food

,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Year,236.1,2626.2,904.9,314.1,223.4,117.7,109.1,196.0,135.9,117.9,...,114.0,104.6,101.1,104.7,102.6,106.7,110.6,110.3,108.2,102.2
January,104.6,405.1,129.0,121.0,121.0,104.0,103.0,102.1,110.3,102.2,...,105.7,101.2,100.8,100.5,101.3,100.7,101.0,101.4,101.3,101.3
February,103.1,129.8,124.8,110.6,110.3,102.4,101.4,101.2,104.4,100.5,...,103.3,100.7,100.2,100.4,100.8,100.6,101.2,101.6,100.8,100.8
March,105.2,123.0,118.3,105.8,108.5,102.9,101.4,100.7,102.8,100.1,...,101.6,100.4,100.1,100.5,100.5,101.0,100.8,106.7,100.1,100.2
April,171.8,116.1,116.1,105.6,107.5,101.8,101.0,100.3,102.6,100.3,...,100.3,100.4,100.6,100.4,100.4,101.7,100.8,102.9,100.3,NaN
May,100.5,110.7,116.2,106.8,108.8,101.7,100.8,100.6,102.0,102.2,...,100.1,100.4,100.6,99.9,100.4,100.2,101.0,100.6,99.7,NaN
June,99.7,120.0,122.7,105.6,107.6,101.2,101.5,100.0,101.7,103.3,...,99.6,100.1,101.0,100.4,99.5,100.2,100.6,98.9,100.0,NaN
July,97.2,108.9,122.5,103.8,104.5,100.2,100.8,99.9,103.2,101.8,...,99.7,100.0,99.0,99.7,99.7,99.9,99.5,98.5,100.5,NaN
August,96.6,107.5,122.6,102.7,102.7,98.3,99.1,102.4,100.4,100.3,...,99.3,99.4,98.2,99.6,99.1,99.2,99.5,98.6,99.9,NaN
September,98.1,112.8,123.7,106.9,102.8,99.5,98.6,139.5,100.8,100.6,...,100.4,99.9,99.3,99.9,99.6,99.6,101.0,99.6,100.9,NaN


### 9.7. Индексы потребительских цен на <font color='#AF4035'> непродовольственные товары </font>  по Российской Федерации в 1991-2023гг.

In [90]:
pci_goods_raw = pd.read_excel(pci_month_link, sheet_name='03')

In [91]:
pci_goods_raw.head(20)

,Индексы потребительских цен на непродовольственные товары по Российской Федерации в 1991-2024*)гг.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
0,К содержанию,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"на конец периода, в %",NaN,NaN
2,NaN,1991.0,1992.00,1993.00,1994.00,1995.00,1996.00,1997.00,1998.00,1999.00,...,2015.00,2016.00,2017.00,2018.00,2019.00,2020.00,2021.00,2022,2023.00,2024
3,к концу предыдущего месяца,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,январь,109.0,311.10,116.31,110.70,111.97,102.69,100.99,100.54,106.16,...,103.20,100.67,100.47,100.30,100.64,100.23,100.53,100.67,100.21,100.47
5,февраль,105.9,147.60,122.81,108.92,109.79,102.20,100.59,100.28,104.01,...,102.06,100.78,100.20,100.13,100.26,100.04,100.58,100.78,99.92,100.26
6,март,104.6,141.90,120.62,107.70,109.17,101.86,100.82,100.21,103.20,...,101.43,100.81,100.22,100.16,100.25,100.48,100.72,111.25,100.14,100.27
7,апрель,170.5,125.40,117.82,107.22,108.72,101.37,100.53,100.17,104.03,...,100.88,100.60,100.17,100.37,100.19,100.44,100.66,100.53,100.15,NaN
8,май,105.4,111.80,115.47,105.45,105.56,101.18,100.57,100.08,102.71,...,100.54,100.42,100.15,100.88,100.21,100.25,100.74,99.91,100.34,NaN
9,июнь,102.2,112.10,116.45,105.00,104.58,101.12,100.45,99.99,101.62,...,100.33,100.46,100.11,100.41,100.17,100.34,100.68,99.59,100.42,NaN


In [92]:
pci_goods = transform_pci(pci_goods_raw)

In [93]:
pci_goods.index = ['Year'] + [months_translate(x) for x in pci_goods.index[1:]]

In [94]:
pci_goods

,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Year,310.7,2673.4,741.8,269.0,216.3,117.8,108.1,199.5,139.2,118.5,...,113.6,106.5,102.8,104.1,103.0,104.8,108.6,112.7,106.0,101.0
January,109.0,311.1,116.3,110.7,112.0,102.7,101.0,100.5,106.2,102.2,...,103.2,100.7,100.5,100.3,100.6,100.2,100.5,100.7,100.2,100.5
February,105.9,147.6,122.8,108.9,109.8,102.2,100.6,100.3,104.0,101.3,...,102.1,100.8,100.2,100.1,100.3,100.0,100.6,100.8,99.9,100.3
March,104.6,141.9,120.6,107.7,109.2,101.9,100.8,100.2,103.2,101.4,...,101.4,100.8,100.2,100.2,100.2,100.5,100.7,111.2,100.1,100.3
April,170.5,125.4,117.8,107.2,108.7,101.4,100.5,100.2,104.0,101.5,...,100.9,100.6,100.2,100.4,100.2,100.4,100.7,100.5,100.2,NaN
May,105.4,111.8,115.5,105.4,105.6,101.2,100.6,100.1,102.7,101.1,...,100.5,100.4,100.2,100.9,100.2,100.2,100.7,99.9,100.3,NaN
June,102.2,112.1,116.4,105.0,104.6,101.1,100.4,100.0,101.6,100.8,...,100.3,100.5,100.1,100.4,100.2,100.3,100.7,99.6,100.4,NaN
July,102.8,109.8,119.2,105.0,105.3,100.9,100.4,100.1,101.9,100.8,...,100.4,100.4,100.1,100.1,100.2,100.3,100.8,99.6,100.9,NaN
August,104.1,110.3,121.0,105.6,106.2,101.1,100.6,107.1,102.4,101.4,...,100.8,100.4,100.1,100.2,100.2,100.4,100.8,100.0,101.1,NaN
September,103.9,112.8,122.3,107.4,106.0,101.2,100.8,154.3,102.7,102.0,...,101.1,100.6,100.2,100.4,100.2,100.6,100.6,100.2,101.1,NaN


### 9.8. Индексы потребительских цен на <font color='#5081AA'> услуги </font>  по Российской Федерации в 1991-2023гг.

In [95]:
pci_services_raw = pd.read_excel(pci_month_link, sheet_name='04')

In [96]:
pci_services_raw.head(20)

,Индексы потребительских цен на услуги по Российской Федерации в 1991-2024*)гг.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
0,К содержанию,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"на конец периода, в %",NaN,NaN
2,NaN,1991.0,1992.00,1993.00,1994.00,1995.00,1996.00,1997.00,1998.00,1999.00,...,2015.00,2016.00,2017.00,2018.00,2019.00,2020.00,2021.00,2022,2023.00,2024
3,к концу предыдущего месяца,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,январь,103.2,230.00,150.00,137.72,122.68,108.05,102.34,101.65,104.13,...,102.16,101.03,100.51,100.08,101.13,100.24,100.38,100.76,101.01,100.78
5,февраль,105.2,140.00,132.00,121.52,119.34,105.64,103.59,101.04,103.17,...,100.83,100.34,100.28,100.12,100.20,100.37,100.44,101.13,100.72,101.06
6,март,105.5,118.00,128.79,116.82,110.48,104.30,102.48,101.23,101.91,...,100.31,100.06,100.00,100.12,100.13,100.09,100.37,103.99,100.97,100.83
7,апрель,121.5,131.20,140.00,131.82,112.22,105.01,101.58,100.98,103.13,...,100.02,100.25,100.16,100.32,100.21,100.12,100.22,101.07,100.79,NaN
8,май,103.9,117.80,134.80,112.72,111.15,102.16,101.97,101.05,102.14,...,100.50,100.46,100.35,100.35,100.39,100.46,100.44,99.68,101.13,NaN
9,июнь,103.5,139.10,117.30,111.41,108.73,101.13,101.00,100.56,103.48,...,100.97,100.56,100.68,100.74,100.60,100.12,100.76,100.88,100.83,NaN


In [97]:
pci_services = transform_pci(pci_services_raw)

In [98]:
pci_services.index = ['Year'] + [months_translate(x) for x in pci_services.index[1:]]

In [99]:
pci_services

,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Year,178.8,2220.5,2411.2,622.4,332.2,148.4,122.5,118.3,134.0,133.7,...,110.2,104.9,104.4,103.9,103.8,102.7,105.0,113.2,108.3,102.7
January,103.2,230.0,150.0,137.7,122.7,108.0,102.3,101.6,104.1,103.4,...,102.2,101.0,100.5,100.1,101.1,100.2,100.4,100.8,101.0,100.8
February,105.2,140.0,132.0,121.5,119.3,105.6,103.6,101.0,103.2,103.0,...,100.8,100.3,100.3,100.1,100.2,100.4,100.4,101.1,100.7,101.1
March,105.5,118.0,128.8,116.8,110.5,104.3,102.5,101.2,101.9,101.4,...,100.3,100.1,100.0,100.1,100.1,100.1,100.4,104.0,101.0,100.8
April,121.5,131.2,140.0,131.8,112.2,105.0,101.6,101.0,103.1,102.1,...,100.0,100.2,100.2,100.3,100.2,100.1,100.2,101.1,100.8,NaN
May,103.9,117.8,134.8,112.7,111.2,102.2,102.0,101.0,102.1,101.3,...,100.5,100.5,100.4,100.4,100.4,100.5,100.4,99.7,101.1,NaN
June,103.5,139.1,117.3,111.4,108.7,101.1,101.0,100.6,103.5,103.0,...,101.0,100.6,100.7,100.7,100.6,100.1,100.8,100.9,100.8,NaN
July,104.4,120.8,129.3,113.3,109.7,102.3,102.3,101.2,103.1,103.8,...,103.0,101.6,101.6,101.3,100.9,101.0,100.9,101.4,100.5,NaN
August,101.9,110.5,149.8,109.9,107.9,102.7,101.0,101.2,101.9,103.0,...,101.3,100.3,100.4,100.3,100.2,100.4,100.3,100.0,99.7,NaN
September,101.8,110.9,124.8,113.4,107.1,101.6,101.2,103.4,102.0,102.8,...,100.0,100.1,100.1,100.2,99.8,99.6,100.0,100.5,100.6,NaN


##### <font color='#820A22'> Export Dataset to Excel

In [100]:
datasets_list = [
    pci, pci_month, pci_food, pci_goods, pci_services
]

In [101]:
sheet_names_list = [
    'ИПЦ (регионы 1992-2022)',
    'ИПЦ Товары и услуги (мес.)', 'ИПЦ Питание (мес.)',
    'ИПЦ Товары (мес.)', 'ИПЦ Услуги (мес.)'
]

In [104]:
for dataset, sheet in zip(datasets_list, sheet_names_list):
    saveit_excel(
    data=dataset,
    filename=dir,
    path=save_excel,
    sheet=sheet
)

'ИПЦ (регионы 1992-2022)' sheet created if file '009.4-009.8_PI-Consumers-Indexes.xlsx'
'ИПЦ Товары и услуги (мес.)' sheet created if file '009.4-009.8_PI-Consumers-Indexes.xlsx'
'ИПЦ Питание (мес.)' sheet created if file '009.4-009.8_PI-Consumers-Indexes.xlsx'
'ИПЦ Товары (мес.)' sheet created if file '009.4-009.8_PI-Consumers-Indexes.xlsx'
'ИПЦ Услуги (мес.)' sheet created if file '009.4-009.8_PI-Consumers-Indexes.xlsx'


In [105]:
saveit_excel(
    data=pci_month_rus,
    filename=dir+'_RUS',
    path=save_excel,
    sheet='ИПЦ (регионы 1992-2022)'
)

'ИПЦ (регионы 1992-2022)' sheet created if file '009.4-009.8_PI-Consumers-Indexes_RUS.xlsx'


### Save Dataset

In [106]:
economics_data['9.4'] = pci

In [107]:
economics_data['9.5'] = pci_month

In [108]:
economics_data['9.6'] = pci_food

In [109]:
economics_data['9.7'] = pci_goods

In [110]:
economics_data['9.8'] = pci_services

In [113]:
saveit(
    file=economics_data,
    name='economics_data',
    dir=save_df
)

File 'economics_data' saved in directory '/Users/achasovsky/Library/CloudStorage/GoogleDrive-alexanderchasovsky@gmail.com/Мой диск/Data Science Projects/# economy-rus/data//00-Datasets/'


### Save Session

In [114]:
save_session(dir)

### Execution time

In [115]:
print(f'Execution time: {stopwatch.stop(start)}')

Execution time: 0:02:31
